In [48]:
import os
import pandas as pd
import re

In [49]:
base_dir = "../data/"

In [50]:
df = pd.read_json(os.path.join(base_dir, "full_compared_df.json"))

In [51]:
df.shape[0]

1182

In [52]:
df.columns

Index(['size', 'connections', 'read_percent', 'write_percent',
       'write_query_id', 'read_query_id', 'template', 'queries_no_p',
       'tps_no_p', 'process_file_no_p', 'queries_p', 'tps_p', 'process_file_p',
       'count_in_group'],
      dtype='object')

In [53]:
df_double = df.loc[df["count_in_group"] > 1 & df["count_in_group"].notna()]

In [54]:
df_double.shape[0]

1166

In [55]:
df_double.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1166 entries, 0 to 1181
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   size               1166 non-null   int64 
 1   connections        1166 non-null   int64 
 2   read_percent       1166 non-null   int64 
 3   write_percent      1166 non-null   int64 
 4   write_query_id     1166 non-null   int64 
 5   read_query_id      1166 non-null   int64 
 6   template           1166 non-null   object
 7   queries_no_p       1166 non-null   object
 8   tps_no_p           1166 non-null   object
 9   process_file_no_p  1166 non-null   object
 10  queries_p          1166 non-null   object
 11  tps_p              1166 non-null   object
 12  process_file_p     1166 non-null   object
 13  count_in_group     1166 non-null   int64 
dtypes: int64(7), object(7)
memory usage: 136.6+ KB


## формирую фичи эффективности (делаю целевым классом не эффективность)

In [56]:
df_double["not_effective"] = (df_double["tps_p"].astype(float) < df_double["tps_no_p"].astype(float)).astype(int)

/tmp/ipykernel_20436/1367067573.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_double["not_effective"] = (df_double["tps_p"].astype(float) < df_double["tps_no_p"].astype(float)).astype(int)


In [57]:
df_double["not_effective"].value_counts(dropna=False)

not_effective
0    790
1    376
Name: count, dtype: int64

## добавляю фичи памяти

**Скажем так, у меня было включено кэша на 2 гб на инстансах бд и 3 гб на gppool в каждом тесте**

In [58]:
df_double["size"].value_counts(dropna=False)

size
300    228
100    218
120    212
280    212
150    192
200     95
400      9
Name: count, dtype: int64

In [59]:
maping_size2memory_consumption = {
    "100": 1.3,
    "120": 1.55,
    "150": 1.95,
    "200": 2.57,
    "280": 3.6,
    "300": 3.85,
    "400": 5.15
}

def maper_func_memory(size, cache_size):
    real_mem = maping_size2memory_consumption[str(int(size))]
    return  cache_size / real_mem

**В этом датасете коэффициент кэшей считается в обратную сторону**

In [60]:
df_double["db_mem_coef"] = df_double["size"].apply(maper_func_memory, cache_size=2.0)

/tmp/ipykernel_20436/2767342478.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_double["db_mem_coef"] = df_double["size"].apply(maper_func_memory, cache_size=2.0)


In [61]:
df_double["pgpool_cache_coef"] = df_double["size"].apply(maper_func_memory, cache_size=3.0)

/tmp/ipykernel_20436/3971481813.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_double["pgpool_cache_coef"] = df_double["size"].apply(maper_func_memory, cache_size=3.0)


In [62]:
df_double[["db_mem_coef","pgpool_cache_coef"]].describe()

,db_mem_coef,pgpool_cache_coef
count,1166.000000,1166.000000
mean,0.960121,1.440181
std,0.395153,0.592729
min,0.388350,0.582524
25%,0.555556,0.833333
50%,1.025641,1.538462
75%,1.290323,1.935484
max,1.538462,2.307692


## Формирую признак сложности

In [63]:
df_double["queries_no_p"].iloc[0]

[{'query': 'SELECT a.*, b.*, t.* FROM pgbench_accounts AS a JOIN pgbench_branches AS b ON a.bid = b.bid JOIN pgbench_tellers AS t ON b.bid = t.bid WHERE a.aid = :aid;',
  'analyze': 'SELECT a.*, b.*, t.* FROM pgbench_accounts AS a JOIN pgbench_branches AS b ON a.bid = b.bid JOIN pgbench_tellers AS t ON b.bid = t.bid WHERE a.aid = 10;',
  'explain': 'cost=8.61..29.05 rows=10 width=813',
  'explain_file': './data/output/explain_id_24_query_id_0.txt'}]

In [64]:
def get_explain_info(row):
    queries_dict = row["queries_no_p"]
    read_id = row["read_query_id"]
    write_id = row ["write_query_id"]
    read_exp = 0.0
    write_exp = 0.0
    try:
        read_query_dict = queries_dict[read_id]
        match = re.search(r"\.\.(\d+\.\d+)", read_query_dict["explain"])
        if match:
            read_exp = float(match.group(1))
    except IndexError:
        pass
    try:
        write_query_dict = queries_dict[write_id]
        match = re.search(r"\.\.(\d+\.\d+)", write_query_dict["explain"])
        if match:
            write_exp = float(match.group(1))
    except IndexError:
        pass
    return read_exp, write_exp

In [65]:
df_double["read_cost"], df_double["write_cost"] = zip(
    *df_double[["queries_no_p","read_query_id","write_query_id"]].apply(get_explain_info, axis=1)
)

/tmp/ipykernel_20436/3448010084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_double["read_cost"], df_double["write_cost"] = zip(
/tmp/ipykernel_20436/3448010084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_double["read_cost"], df_double["write_cost"] = zip(


In [66]:
df_double[["read_cost", "write_cost"]].describe()

,read_cost,write_cost
count,1166.000000,1166.000000
mean,427.778328,1395.123842
std,1765.437988,7524.922895
min,0.010000,0.000000
25%,18.410000,8.460000
50%,32.550000,13.090000
75%,61.857500,289.272500
max,34414.940000,100081.720000


## Записываю фичи в csv

In [67]:
df_double.columns

Index(['size', 'connections', 'read_percent', 'write_percent',
       'write_query_id', 'read_query_id', 'template', 'queries_no_p',
       'tps_no_p', 'process_file_no_p', 'queries_p', 'tps_p', 'process_file_p',
       'count_in_group', 'not_effective', 'db_mem_coef', 'pgpool_cache_coef',
       'read_cost', 'write_cost'],
      dtype='object')

In [68]:
df_need = df_double[['connections', 'read_percent', 'write_percent',
                     'db_mem_coef', 'pgpool_cache_coef', 'read_cost', 'write_cost',
                     'not_effective']]

In [69]:
df_need.columns

Index(['connections', 'read_percent', 'write_percent', 'db_mem_coef',
       'pgpool_cache_coef', 'read_cost', 'write_cost', 'not_effective'],
      dtype='object')

In [70]:
df_need.head()

,connections,read_percent,write_percent,db_mem_coef,pgpool_cache_coef,read_cost,write_cost,not_effective
0,80,100,0,1.538462,2.307692,29.05,0.0,0
1,90,100,0,1.538462,2.307692,29.05,0.0,0
2,100,100,0,1.538462,2.307692,29.05,0.0,0
3,150,100,0,1.538462,2.307692,29.05,0.0,0
4,210,100,0,1.538462,2.307692,29.05,0.0,0


In [71]:
df_need.to_csv(os.path.join(base_dir, "prepared_df2.csv"), index=False)